In [1]:
import numpy as np
import networkx as nx
import random
import torch
from train import Train_Model
#from tqdm import tqdm, trange
from tqdm import tqdm_notebook as tqdm
import gc
from model import *
from utils import *

print(torch.__version__)
print(torch.cuda.is_available())

1.7.1+cpu
False


In [ ]:
def similarity_filter(similarity_matrix, node, value=1.0, direction=1):
    """
    返回与节点相关性最高/最低的邻接节点
    """
    pass

In [2]:
def pre_sample_for_ether(walk_times,adj_sparse, train_index, batch_size, save_name, do_walk):
    if do_walk:
        nodes_num = len(train_index)
        walks = torch.zeros(nodes_num, walk_times+1).long().cuda()
        batches = create_batches_forList(train_index, batch_size, True)
        i=0
        degrees = torch.tensor(adj_sparse.sum(1)).view(-1).cuda()
        candi_node = 0
        for batch in batches:
            walks[i*batch_size : i*batch_size + len(batch), 0] = torch.tensor(batch).cuda()
            candi_node = sparse_mx_to_torch_sparse_tensor(adj_sparse[batch.cpu().numpy()]).cuda()
            chosen_node = torch.zeros(len(batch), adj_sparse.shape[0]).cuda()
            for id in range(len(batch)):
                chosen_node[id][batch[id]] = 1.
            candi_node = ((- chosen_node + candi_node)> 0.0).float()
            for walk_id in range(walk_times):
                for x in range(candi_node.shape[0]):
                    if candi_node[x].sum()==0:
                        candi_node[x][batch[x]] = 1.

                p = candi_node
                new_node = torch.multinomial(p,1).squeeze(1)
                walks[i*batch_size : i*batch_size + len(batch), walk_id+1] = new_node
                for id in range(len(batch)):
                    chosen_node[id][new_node[id]] = 1.
                candi_node = candi_node - chosen_node + sparse_mx_to_torch_sparse_tensor(adj_sparse[new_node.cpu()]).cuda()
                candi_node = (candi_node> 0.0).float()
                
            i+=1
        torch.cuda.empty_cache()
        result1 = np.array(walks.cpu())
        io.savemat(save_name+'.mat',{save_name:result1})
        return walks
    else:
        walks = io.loadmat(save_name+'.mat')
        print("walks",walks)
        return torch.tensor(walks[save_name]).cuda()

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
sparse_adj, sparse_adj_train,sparse_adj_train_all,features, train_feature,train_feature_all,labels, train_labels, id_train, id_valid, id_test, num_labels = Origin_load_ether_data()
sparse_adj.setdiag(1.0) 
sparse_adj_train.setdiag(1.0) 
sparse_adj_train_all.setdiag(1.0) 

features = (1402220, 12) type = <class 'numpy.ndarray'>


AssertionError: Torch not compiled with CUDA enabled

In [4]:
for walk in range(1,10):
    for seed in range(10,20):
        torch.manual_seed(seed)
        random.seed(seed)
        torch.cuda.manual_seed(seed)
        np.random.seed(seed)
        save_name = 'seed='+str(seed)+'_walk='+str(walk)
        print('process...'+'train_'+save_name)
        pre_sample_for_ether(walk,sparse_adj_train_all, id_train, 32, './walks_ether_random/train_'+save_name, False)
        print('process...'+'valid_'+save_name)
        pre_sample_for_ether(walk,sparse_adj, id_valid, 32, './walks_ether_random/valid_'+save_name, False)
        print('process...'+'test_'+save_name)
        pre_sample_for_ether(walk,sparse_adj, id_test, 32, './walks_ether_random/test_'+save_name, False)

process...train_seed=10_walk=1


NameError: name 'sparse_adj_train_all' is not defined